In [1]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import numpy as np
from shapely.ops import nearest_points
import requests

In [2]:
ruta1000 = geopandas.read_file('/home/ai6644/Malmo/Data/Sjöbo/GIS/B1 ruta 1000 difference with 250.shp')
ruta1000 = ruta1000.to_crs("EPSG:4326")

ruta250 = geopandas.read_file('/home/ai6644/Malmo/Data/Sjöbo/GIS/B1 ruta 250.shp')
ruta250 = ruta250.to_crs("EPSG:4326")

rutas = geopandas.GeoDataFrame(pd.concat([ruta250, ruta1000], axis=0))

In [3]:
rutas['Ruta']= rutas.Ruta.astype(int)

In [4]:
rutas = rutas.set_index('Ruta')

In [5]:
class Coord(object):
    """Coordinate.

    Parameters
    ----------
    lat : <float> latitude
    lon : <float> longitude
    latlon : <list> list with both lat and long. Latitude first!
    """
    def __init__(self, lat=None, lon=None, latlon=None):
        if latlon is not None:
            if len(latlon) != 2:
                raise Exception("Wrong coordinate latlon format. Should be a list of two floats.")
            self.lat = latlon[0]
            self.lon = latlon[1]
        elif lat is None or lon is None:
            raise Exception("Coordinates not provided")
        else:
            self.lat = lat
            self.lon = lon

    def to_json(self):
        return json.dumps(self, default=lambda o: self._try(o), sort_keys=True, indent=4, separators=(',', ':'))

    @staticmethod
    def _try(o):
        try:
            if o.__class__ == Coord:
                raise Exception()
            return o.__dict__
        except:
            return str(o)

    def __str__(self):
        return str(self.lat) + ',' + str(self.lon)

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.lat == other.lat and self.lon == other.lon

    def __hash__(self):
        return hash((self.lat, self.lon))


In [6]:
def _osrm_tdm_request(coords, sources, destinations):
    url_coords = ';'.join([str(coord.x) + ',' + str(coord.y) for coord in coords])
    url_sources = 'sources=' + ';'.join([str(s) for s in sources])
    url_destinations = 'destinations=' + ';'.join([str(d) for d in destinations])
    url_server = 'http://0.0.0.0:5000/table/v1/driving/'
    url_options = 'fallback_speed=9999999999&annotations=duration,distance'
    url_full = '{}{}?{}&{}&{}'.format(url_server, url_coords, url_options, url_sources, url_destinations)
    resp = requests.get(url=url_full)

    jresp = resp.json()
    if jresp.get('code') != 'Ok':
        log.error(jresp.get('code'))
        log.error(jresp.get('message'))
        resp.raise_for_status()

    return jresp.get('durations'), jresp.get('distances')

In [7]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [8]:
rs = rutas
slice_size = 1000

time_row_df = None
distance_row_df = None
time_big_df = None
distance_big_df = None

for sources in chunks(range(len(rutas.index)), slice_size):
    time_row_df = None
    distance_row_df = None
    for destinations in chunks(range(len(rutas.index)), slice_size):
#         tdm = _osrm_tdm_request(rs.centroid, sources, destinations)
        tdm = (np.zeros((len(sources), len(destinations))), np.zeros((len(sources), len(destinations))))
        time = pd.DataFrame(tdm[0], index=sources, columns=destinations)
        distance = pd.DataFrame(tdm[1], index=sources, columns=destinations)
        
        if time_row_df is None:
            time_row_df = time
            distance_row_df = distance
        else:
            time_row_df = pd.concat([time_row_df, time], axis = 1)
            distance_row_df = pd.concat([distance_row_df, distance], axis = 1)
    
    if time_big_df is None:
        time_big_df = time_row_df
        distance_big_df = distance_row_df
    else:
        time_big_df = pd.concat([time_big_df, time_row_df], axis = 0)
        distance_big_df = pd.concat([distance_big_df, distance_row_df], axis = 0)

KeyboardInterrupt: 

In [ ]:
time_big_df.columns = rutas.index.astype('string')
distance_big_df.columns = rutas.index.astype('string')
# time_big_df.index = rutas.index
# distance_big_df.index = rutas.index

In [ ]:
time_big_df.to_feather('time_matrix.feather')
distance_big_df.to_feather('distance_matrix.feather')